In [1]:
def query(sqlString:str)->None:
    
    try:
        return cxn.query(sqlString).show(max_width = 100000, max_rows = 100000)
    except Exception as e:
        return e


In [2]:
def getInvoices():
    
    try:
        relativeInvoicesURL = '/v1/invoices'
        response = json.loads(
        rs.get(
            f"{base_url}{relativeInvoicesURL}",
            headers=HTTPheaders)
        .content)

        return response['items']
    
    except Exception as e:
        return e

In [3]:
def getOneTimeInvoiceLineItems(invoiceID:str)->dict:
    
    try:
        newCommerceOneTimeBillingURL = f'/v1/invoices/OneTime-{invoiceID}/lineitems/OneTime/BillingLineItems?size=5000'

        response = json.loads(
            rs.get(
                    f"{base_url}{newCommerceOneTimeBillingURL}",
                    headers=HTTPheaders)
            .content) if rs.get(
                    f"{base_url}{newCommerceOneTimeBillingURL}",
                    headers=HTTPheaders).status_code == 200 else None # if response status code = 200
        
        return response['items']
    
    except Exception as e:
        return e

In [4]:

import requests as rs
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
from azure.storage.blob import ContainerClient,BlobServiceClient
from io import StringIO
import numpy as np
import json
import datetime as dt
import warnings
import duckdb as db
import time

# Set display options

pd.set_option('max_colwidth', None)

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)

cxn = db.connect('../DB/dbfile')

print( 'connected to Db')
time.sleep(1)

# Parse Secrets

try:
    secrets = json.load(open(file='../secrets.json'))
    print('Secrets file found.')
except:
    print('Secrets file not found or some error in file. Make sure the file exists in the same directory as the code.')

if secrets:
    refresh_token = secrets['refresh_token']

    app_id = secrets['app_id']

    app_secret = secrets['app_secret']

    request_body = {
            "grant_type": "refresh_token",
            "refresh_token": refresh_token,
            "scope": "openid",
            "resource": "https://api.partnercenter.microsoft.com",
            "client_id": app_id,
            "client_secret": app_secret,
        }

    base_url = (
    'https://api.partnercenter.microsoft.com'
    )


    isAccessTokenObtained = 0


try:
    req = rs.post(
                "https://login.windows.net/6e75cca6-47f0-47a3-a928-9d5315750bd9/oauth2/token"
                ,data = request_body)
    
    access_token = json.loads(req.text)['access_token']
    isAccessTokenObtained = 1
    print('Refresh Token valid, access token obtained. \nAccess token : ',access_token[:20]+'...')
except:
    print('Refresh token expired, or no internet connection. \nUnable to obtain access token')
    print(json.loads(req.text))
    

if isAccessTokenObtained == 1:
        
        HTTPheaders = {'Authorization': 'Bearer ' + access_token}


connected to Db
Secrets file found.
Refresh Token valid, access token obtained. 
Access token :  eyJ0eXAiOiJKV1QiLCJh...


In [5]:
latestInvoice = cxn.query("select id from invoices where id like 'G%' order by invoicedate desc limit 1 ")
latestInvoice = str(latestInvoice).replace("┌────────────┐\n│     id     │\n│  varchar   │\n├────────────┤\n│ ",'').replace(" │\n└────────────┘\n",'')
latestInvoice

'G038562576'

In [6]:
previousInvoice = cxn.query("select id from invoices where id like 'G%' order by invoicedate  desc limit 1 offset 1 ")
previousInvoice = str(previousInvoice).replace("┌────────────┐\n│     id     │\n│  varchar   │\n├────────────┤\n│ ",'').replace(" │\n└────────────┘\n",'')
previousInvoice

'G036230777'

In [7]:
latestMonthDF = pd.DataFrame(getOneTimeInvoiceLineItems(latestInvoice))
latestMonthDF.head()

,partnerId,customerId,customerName,customerDomainName,customerCountry,invoiceNumber,mpnId,resellerMpnId,orderId,orderDate,productId,skuId,availabilityId,productName,skuName,productQualifiers,chargeType,unitPrice,effectiveUnitPrice,unitType,quantity,subtotal,taxTotal,totalForCustomer,currency,publisherName,publisherId,subscriptionDescription,subscriptionId,subscriptionStartDate,subscriptionEndDate,chargeStartDate,chargeEndDate,termAndBillingCycle,alternateId,referenceId,priceAdjustmentDescription,discountDetails,pricingCurrency,pcToBCExchangeRate,pcToBCExchangeRateDate,billableQuantity,meterDescription,billingFrequency,reservationOrderId,invoiceLineItemType,billingProvider,promotionId,attributes
0,6e75cca6-47f0-47a3-a928-9d5315750bd9,86fb359e-1360-4ab3-b90d-2a68e8c007b9,Data Semantics Pvt Ltd,DSglobal.onmicrosoft.com,IN,G038562576,2240443,0,i865EsrwBdGpAO_xbJyBQLW1kiHa6n5-1,2024-01-02T09:38:43.4669373Z,DZH318Z09HH9,0016,DZH318Z0B3RM,Twilio SendGrid,Essentials 50K,[],renew,1530.01357,1530.010000,,1,1530.01,275.40,1805.41,INR,SendGrid,18197210,account-receivable,2fec0c84-27e3-4e4e-cf1b-05620a8069f4,2024-01-02T00:00:00Z,2024-02-01T00:00:00Z,2024-01-02T00:00:00Z,2024-02-01T00:00:00Z,1 Month Subscription,b95a51b179d9,64b23543-0d99-4c7b-a639-16ae03f165aa,"[""Price for given term""]",,INR,1.00000,2024-01-01T00:00:00,1.000000,,,2fec0c84-27e3-4e4e-cf1b-05620a8069f4,billing_line_items,one_time,,{'objectType': 'OneTimeInvoiceLineItem'}
1,6e75cca6-47f0-47a3-a928-9d5315750bd9,a283c257-9b2b-432c-a382-127ad270b2a5,Brio Technologies Pvt Ltd,pocbrio.onmicrosoft.com,IN,G038562576,2240443,0,qrJ0C5uf2WhFxyjh026MR52fPP-VLEc41,2024-02-03T13:12:56.2071157Z,DZH318Z0BPVM,001B,DZH318XZMBKS,Virtual Machines Dv2 Series Windows,Virtual Machines Dv2 Series Windows - D1 v2 - IN Central,[],new,0.13000,0.110500,1 Hour,1,125.86,22.65,148.51,INR,Microsoft,,Azure plan,84c8390a-da8b-7623-44aa-e86f73ccb9c4,NaN,NaN,2024-01-01T00:00:00Z,2024-01-31T23:59:59Z,1 Compute Hour,8c8b146c700f,e841c3d2-6190-4978-9216-9cbb08e85017,"[""15.0% Partner earned credit for services managed""]",,USD,83.35125,2024-01-31T23:59:59Z,13.683427,Virtual Machines Dv2/DSv2 Series Windows - D1 v2/DS1 v2 Hours - IN Central,,6d0fca2558a1e3b8ebdccedeb767e1ef68a633b0bd06290794dbc18318824848,billing_line_items,one_time,,{'objectType': 'OneTimeInvoiceLineItem'}
2,6e75cca6-47f0-47a3-a928-9d5315750bd9,d95edf1e-2c7d-42ee-8e84-163229f87133,PurpleTalk,purpletalk.onmicrosoft.com,IN,G038562576,2240443,0,hIqjNfP9Mwj5v6Y0H6bVSKgKzOQkZB_c1,2024-02-03T16:28:33.0878308Z,DZH318Z0BQDW,0005,DZH318Z0B4SZ,SQL Database Single/Elastic Pool General Purpose - Storage,SQL Database Single/Elastic Pool General Purpose - Storage - US East,[],new,0.11500,0.097750,1 GB/Month,1,0.00,0.00,0.00,INR,Microsoft,,Azure plan,7fc27f03-8633-aa9a-22ee-89ac92156291,NaN,NaN,2024-01-01T00:00:00Z,2024-01-31T23:59:59Z,Data Stored (GB/Month),aeb6d25e2164,6816d947-5863-4932-926a-a125e91b5868,"[""15.0% Partner earned credit for services managed""]",,USD,83.35125,2024-01-31T23:59:59Z,0.017473,SQL Database Single/Elastic Pool General Purpose - Storage - Data Stored GB/Month - US East,,545351f765efab533d56253c75b3659dcbc237fca19865429d79141632929ef2,billing_line_items,one_time,,{'objectType': 'OneTimeInvoiceLineItem'}
3,6e75cca6-47f0-47a3-a928-9d5315750bd9,e656f7a7-923b-4565-840b-bd9c18f4fd2d,BridgeLabz Solutions Pvt Lltd,adminbridgelabz.onmicrosoft.com,IN,G038562576,2240443,0,wKTUj7MNGO1fQ_Rx1vg-SLHE0bXYbv7K1,2024-02-04T06:23:04.6342044Z,DZH318Z0BNZH,006F,DZH318Z0QCXV,General Block Blob,General Block Blob - LRS,[],new,0.00036,0.000306,10K,1,0.00,0.00,0.00,INR,Microsoft,,Azure plan,f774d360-8863-478a-c309-6d90733168b1,NaN,NaN,2024-01-01T00:00:00Z,2024-01-31T23:59:59Z,10K List and Create Container Operations,58b487f58e0a,d011cf49-129d-4c35-bfca-cb1a48614d8a,"[""15.0% Partner earned credit for services managed""]",,USD,83.35125,2024-01-31T23:59:59Z,0.900900,General Block Blob - LRS - 10K List and Create Container Operations,,b5faa12fec31f19d572e9bf5e

In [8]:
previousMonthDF = pd.DataFrame(getOneTimeInvoiceLineItems(previousInvoice))
previousMonthDF.head()

,partnerId,customerId,customerName,customerDomainName,customerCountry,invoiceNumber,mpnId,resellerMpnId,orderId,orderDate,productId,skuId,availabilityId,productName,skuName,productQualifiers,chargeType,unitPrice,effectiveUnitPrice,unitType,quantity,subtotal,taxTotal,totalForCustomer,currency,publisherName,publisherId,subscriptionDescription,subscriptionId,chargeStartDate,chargeEndDate,termAndBillingCycle,alternateId,referenceId,priceAdjustmentDescription,discountDetails,pricingCurrency,pcToBCExchangeRate,pcToBCExchangeRateDate,billableQuantity,meterDescription,billingFrequency,reservationOrderId,invoiceLineItemType,billingProvider,promotionId,attributes,subscriptionStartDate,subscriptionEndDate
0,6e75cca6-47f0-47a3-a928-9d5315750bd9,d4815a57-d729-404d-acf8-c4da4e8b875a,Tata Chemicals Limited,tatachemicals.net,IN,G036230777,2240443,0,tUvDFTHutf0yA_Q0KT_zQYVKFn2K6yM31,2024-01-03T21:21:09.949796Z,DZH318Z0BNX0,0021,DZH318Z09T1W,Azure DNS,Azure DNS - Private,[],new,0.50000,0.425000,1,1,176.68,31.80,208.48,INR,Microsoft,,Azure plan,392186e1-5981-4d72-c052-adfbb3645f4c,2023-12-01T00:00:00Z,2023-12-31T23:59:59Z,1 Zone,8aa81e096424,9f2f4780-c952-4ddf-b2ec-599a14380516,"[""15.0% Partner earned credit for services managed""]",,USD,83.34375,2023-12-31T23:59:59Z,5.000000,Azure DNS - Private 1 Zone,,ccf39bbb5280df08c54c507aa85a0b4e203f108b8ca730e2e0949bc7edd5bfed,billing_line_items,one_time,,{'objectType': 'OneTimeInvoiceLineItem'},NaN,NaN
1,6e75cca6-47f0-47a3-a928-9d5315750bd9,b05d8285-01a8-496a-9ee6-b0440e69121c,Aapkapainter Solutions Pvt Ltd,anupamaapkapainter.onmicrosoft.com,IN,G036230777,2240443,0,9TU9-mcxSXgpz5ymp720QZ29Op25EUio1,2024-01-03T18:17:05.0117523Z,DZH318Z0BP05,006J,DZH318Z0GM03,Backup,Backup - Standard - US East,[],new,0.04480,0.038080,1 GB/Month,1,105.84,19.05,124.89,INR,Microsoft,,Azure plan,9ef9ff61-dbdd-4fbc-de83-def3ef5f6c4a,2023-12-01T00:00:00Z,2023-12-31T23:59:59Z,GRS Data Stored (GB/Month),d5aaf51fa315,4e057160-bdce-43fb-88d5-edc3a65957b3,"[""15.0% Partner earned credit for services managed""]",,USD,83.34375,2023-12-31T23:59:59Z,33.404000,Backup - GRS Data Stored (GB/Month) - US East,,5912feac0cf2462643c7b809e4bab974969000790598fe75788bddc30f1ba8b7,billing_line_items,one_time,,{'objectType': 'OneTimeInvoiceLineItem'},NaN,NaN
2,6e75cca6-47f0-47a3-a928-9d5315750bd9,07782621-ac3c-431b-b38a-b596b49956c0,briodevptm,briodevptm.onmicrosoft.com,IN,G036230777,2240443,0,K0VAxK9tfbZ7ptkHTim5T73od_vhVuSL1,2024-01-03T19:55:59.5947268Z,DZH318Z0BNZH,006F,DZH318Z0QCXV,General Block Blob,General Block Blob - LRS,[],new,0.00036,0.000306,10K,1,0.00,0.00,0.00,INR,Microsoft,,Azure plan,b68b5c74-15d1-4edb-c5c4-5261052d03c6,2023-12-01T00:00:00Z,2023-12-31T23:59:59Z,10K List and Create Container Operations,cbca96c848d6,440dabb5-6115-470b-8367-d67d8a4bad19,"[""15.0% Partner earned credit for services managed""]",,USD,83.34375,2023-12-31T23:59:59Z,0.002000,General Block Blob - LRS - 10K List and Create Container Operations,,035c679b6bc81ee69dddd9c3f9fb28db08b1fd00c4bbb07510a3dc2faa760d31,billing_line_items,one_time,,{'objectType': 'OneTimeInvoiceLineItem'},NaN,NaN
3,6e75cca6-47f0-47a3-a928-9d5315750bd9,b04c2da1-cb08-4b2a-9099-51b492037918,Intelogixs Research Private Limited,intelogixs.com,IN,G036230777,2240443,0,mNvOalIJyqULGlcHqpQBTZ5mKqwTN1pm1,2024-01-03T19:38:06.5629395Z,DZH318Z0BP0L,00C4,DZH318Z0L2LH,Standard HDD Managed Disks,Standard HDD Managed Disks - S10 LRS - US East,[],new,5.88800,5.004800,1/Month,1,3652.95,657.53,4310.48,INR,Microsoft,,Azure plan,22b7da32-0ed2-4df4-d549-4137ba6afe95,2023-12-01T00:00:00Z,2023-12-31T23:59:59Z,1 Disk/Month,2c074816288d,9149ec0c-6769-4d9a-a8ae-094ad1947c9b,"[""15.0% Partner earned credit for services managed""]",,USD,83.34375,2023-12-31T23:59:59Z,8.758848,Standard HDD Managed Disks - S10 - LRS - 1/Month Disk - US East,,cedeb1c276d69482d467299d026a08fb3dd97c13925d9edf193eb62d004dbf34,billing_line_items,one_time,,{'objectType': 'OneTimeInvoiceLineItem'},NaN,NaN
4,6e75cca6-47f0-47a3-a928-9d5315750bd9,76

In [139]:
requiredColumns = """customerDomainName,orderDate,subscriptionid,date_trunc('day',cast(chargestartdate as date)) truncatedchargestartdate,
chargeenddate,chargetype,effectiveunitprice,billablequantity,totalforcustomer,monthname(cast(subscriptionstartdate as datetime)) mName"""

In [140]:
query(f"""select {requiredColumns} from latestMonthDF where productid like 'CFQ%'
       order by 
      customerdomainname,subscriptionid
      """)

┌───────────────────────────────────────┬──────────────────────────────┬──────────────────────────────────────┬──────────────────────────┬──────────────────────┬─────────────────┬────────────────────┬──────────────────┬──────────────────┬───────────┐
│          customerDomainName           │          orderDate           │            subscriptionId            │ truncatedchargestartdate │    chargeEndDate     │   chargeType    │ effectiveUnitPrice │ billableQuantity │ totalForCustomer │   mName   │
│                varchar                │           varchar            │               varchar                │           date           │       varchar        │     varchar     │       double       │      double      │      double      │  varchar  │
├───────────────────────────────────────┼──────────────────────────────┼──────────────────────────────────────┼──────────────────────────┼──────────────────────┼─────────────────┼────────────────────┼──────────────────┼──────────────────┼─────────

In [132]:
cxn.query(f"""create or replace temp table latestTbl as select {requiredColumns} from latestMonthDF where productid like 'CFQ%'
       order by 
      customerdomainname,subscriptionid
      """)
query('select count(1) from latestTbl')

┌──────────┐
│ count(1) │
│  int64   │
├──────────┤
│     1225 │
└──────────┘



In [133]:
cxn.query(f"""create or replace temp table previousTbl as select {requiredColumns} from previousMonthDF where productid like 'CFQ%'
       order by 
      customerdomainname,subscriptionid
      """)
query('select count(1) from previousTbl')

┌──────────┐
│ count(1) │
│  int64   │
├──────────┤
│     1081 │
└──────────┘



In [134]:
query("select * from latestTbL")

┌───────────────────────────────────────┬──────────────────────────────┬──────────────────────────────────────┬──────────────────────────┬──────────────────────┬─────────────────┬────────────────────┬──────────────────┬──────────────────┬───────────┐
│          customerDomainName           │          orderDate           │            subscriptionId            │ truncatedchargestartdate │    chargeEndDate     │   chargeType    │ effectiveUnitPrice │ billableQuantity │ totalForCustomer │   mName   │
│                varchar                │           varchar            │               varchar                │           date           │       varchar        │     varchar     │       double       │      double      │      double      │  varchar  │
├───────────────────────────────────────┼──────────────────────────────┼──────────────────────────────────────┼──────────────────────────┼──────────────────────┼─────────────────┼────────────────────┼──────────────────┼──────────────────┼─────────

In [436]:
requiredColumns = """
 customerId,
 customerName,
 customerDomainName,
 invoiceNumber,
 orderDate,
 invoiceNumber,
 substring(skuName,0,30) skuName,
 subscriptionId,
 chargeType,
 effectiveUnitPrice,
 quantity,
 subtotal,
 taxTotal,
 totalForCustomer,
 chargeStartDate,
 chargeEndDate,
 referenceId,
 billableQuantity,
 invoiceMonth,
 subscriptionStartDate,
 subscriptionEndDate,
 monthname(cast(subscriptionstartdate as datetime)) || cast(year(cast(subscriptionstartdate as datetime))as varchar) as  mn
 """ # concat monthname and year

# raw

In [455]:
query(f"""
      --
      with t0 as 
      (select {requiredColumns}
      from onetime 
      where 
      productid like 'CFQ%'
      --and customerDomainName = 'yaxiso365.onmicrosoft.com'
      and customerDomainName = 'annet50.onmicrosoft.com'
       
       --and subscriptionId = 'cd963bd4-ada7-4235-c211-0f926c3a7b87'
       order by subscriptionid, orderdate)

       ,

       t1 as 

      (select customerdomainname,subscriptionid,round(sum(totalforcustomer)) sm from t0
       group by 1,2 order by 3 desc)

       select * from t0
       """)

┌──────────────────────────────────────┬──────────────────────────────┬─────────────────────────┬───────────────┬──────────────────────────────┬─────────────────┬───────────────────────────────┬──────────────────────────────────────┬────────────────┬────────────────────┬──────────┬────────────┬───────────┬──────────────────┬──────────────────────────────┬──────────────────────┬──────────────────────────────────────┬──────────────────┬────────────────┬───────────────────────┬──────────────────────┬───────────────┐
│              customerId              │         customerName         │   customerDomainName    │ invoiceNumber │          orderDate           │ invoiceNumber:1 │            skuName            │            subscriptionId            │   chargeType   │ effectiveUnitPrice │ quantity │  subtotal  │ taxTotal  │ totalForCustomer │       chargeStartDate        │    chargeEndDate     │             referenceId              │ billableQuantity │  invoiceMonth  │ subscriptionStartDate │ s

# Transformed

In [486]:
query(f"""
 
with 
      t0 as (
      select {requiredColumns}
      from onetime where productid like 'CFQ%'  
      and customerDomainName = 'yaxiso365.onmicrosoft.com'
      --and customerDomainName = 'annet50.onmicrosoft.com'
      --and subscriptionid = '12266bbd-6d4a-422f-d413-4c688cc48550'
      )
      ,
     
      t1 as                                            /* club rows that have similar charge types and reference IDs. */

      (select customerid,customerdomainname,max(cast(orderDate as datetime)) orderdate1, last(invoicenumber order by orderdate) newInvoiceNumber
      ,subscriptionid,date_trunc('day',cast(chargestartdate as datetime)) truncatedchargestartdate
      , chargeenddate,chargetype
      ,first(billablequantity order by totalforcustomer desc) fbqty , last(billablequantity order by totalforcustomer desc) lbqty
      ,fbqty - lbqty QtyAdded
      ,case when qtyadded = 0 then fbqty else qtyadded end as QtyAdded_adjusted
      ,round(sum(totalforcustomer),2) newTotal
      ,case when newTotal < 0 then -(abs(QtyAdded_adjusted)) else QtyAdded_adjusted end as QtyAdded_adjustedForNegativity
      ,referenceid
      ,mn
      from t0  
      group by customerid,customerdomainname,subscriptionid,truncatedchargestartdate, chargeenddate,chargetype,referenceid,mn
      order by subscriptionid,orderdate1)
      ,

       t2 as

       (select *
      ,sum(QtyAdded_adjustedForNegativity) over(partition by customerid,customerdomainname, subscriptionid, mn ,cast(chargeenddate as date) order by orderdate1 rows between unbounded preceding and  0 following) as finalLicenseQty
      ,truncatedchargestartdate newSDate
      , lead(truncatedchargestartdate) over(partition by customerid, customerdomainname,subscriptionid,mn order by orderdate1 ) adjustedChargeStartDate 
      ,case when adjustedChargeStartDate is null then date_add(cast(chargeenddate as date), interval 1 day) 
      else adjustedChargeStartDate end as  lead_adjustedChargeStartDate,
       from t1
        order by subscriptionid,orderdate1 )
      ,

      subscriptionWiseBreakdown as
       
         (select customerid,customerdomainname,subscriptionid,round(sum(newTotal),2) sm from t2 group by 1,2,3 order by 4 desc)
      ,

      final as 
      (
      select customerid,customerDomainName,newInvoiceNumber as invoiceId,orderDate1 orderDate,subscriptionId,newSdate as chargeStartDate, cast(lead_adjustedChargeStartDate as date) as chargeEndDate,chargeType, finalLicenseQty Qty, newTotal Amount
      --, mn billingMonth
      from t2
      order by subscriptionid,orderdate
      )

      select * from (with f as (select * from final) select * from f)
      
""")

┌──────────────────────────────────────┬───────────────────────────┬────────────┬────────────────────────────┬──────────────────────────────────────┬─────────────────┬───────────────┬─────────────────┬────────┬────────────┐
│              customerId              │    customerDomainName     │ invoiceId  │         orderDate          │            subscriptionId            │ chargeStartDate │ chargeEndDate │   chargeType    │  Qty   │   Amount   │
│               varchar                │          varchar          │  varchar   │         timestamp          │               varchar                │      date       │     date      │     varchar     │ double │   double   │
├──────────────────────────────────────┼───────────────────────────┼────────────┼────────────────────────────┼──────────────────────────────────────┼─────────────────┼───────────────┼─────────────────┼────────┼────────────┤
│ d2781ccb-abb1-410d-969f-b1259cc2a697 │ yaxiso365.onmicrosoft.com │ G018993861 │ 2023-01-31 15:36:07.81

In [207]:
query("""
select customerDomainName ,subscriptionid ,count(1) from onetime where chargetype in ('moveQuantity','convert') and totalforcustomer > 0 
      group by 1,2
      order by 3 desc
""")

┌─────────────────────────────────┬──────────────────────────────────────┬──────────┐
│       customerDomainName        │            subscriptionId            │ count(1) │
│             varchar             │               varchar                │  int64   │
├─────────────────────────────────┼──────────────────────────────────────┼──────────┤
│ annet50.onmicrosoft.com         │ 5e45831b-b5da-4fb0-c743-af024dda5fdb │        4 │
│ annet50.onmicrosoft.com         │ cd963bd4-ada7-4235-c211-0f926c3a7b87 │        4 │
│ annet50.onmicrosoft.com         │ 9b1415e6-c30f-4c18-d365-21b7e22d2bbc │        3 │
│ hela.in                         │ 9960fc68-3d83-4275-c438-0443adf7d025 │        3 │
│ annet50.onmicrosoft.com         │ e1924442-2f57-41be-d2b3-c85edd435368 │        3 │
│ teyro.com                       │ 6849ca74-e04b-4d48-d9c0-6d947d45debf │        1 │
│ brglobal.in                     │ 00e12312-c9a2-4a0e-d2a7-1871541374fb │        1 │
│ annet50.onmicrosoft.com         │ 426f73d1-6c98-416c

In [321]:
query(f"""
      --select customerdomainname,subscriptionid,round(sum(tfc),2) sm from 
    --/*
       (select * exclude (qty,qtyadded,actualqty) , case when chargetype in ('moveQuantity','convert') and tfc <0 then -actualqty else actualqty end as actualqty1
      ,sum(actualqty1) over(partition by subscriptionid,mn order by orderdate rows between unbounded preceding and 0 following )totalqty
      ,truncatedchargestartdate newSDate
      ,case when lead(truncatedchargestartdate) over(partition by subscriptionid order by orderdate ) is null then chargeenddate 
      else lead(truncatedchargestartdate) over(partition by subscriptionid order by orderdate ) end as  newEDate,
       from 
      --*/
(select customerdomainname,max(orderdate) orderdate,subscriptionid,date_trunc('day',cast(chargestartdate as datetime)) truncatedchargestartdate
      , chargeenddate,chargetype
      ,first(effectiveUnitPrice order by effectiveUnitPrice desc) absEffectiveUnitPrice
      ,max(billablequantity) qty
      ,first(billablequantity order by effectiveUnitPrice desc) - last(billablequantity order by effectiveUnitPrice desc) QtyAdded,referenceid
      ,case when chargetype in ('addQuantity','removeQuantity','moveQuantity','convert','cycleCharge') then qtyadded else qty end as actualqty
      ,round(sum(totalforcustomer),2) tfc
      ,monthname(cast(subscriptionstartdate as datetime))||cast(year(cast(subscriptionstartdate as datetime))as varchar) mn
      from onetime 
      where 
      customerDomainName = 'annet50.onmicrosoft.com'
      --and 
      --subscriptionid = '12266bbd-6d4a-422f-d413-4c688cc48550'
      group by customerdomainname,subscriptionid,truncatedchargestartdate, chargeenddate,chargetype,referenceid,mn
      order by subscriptionid,orderdate)
      order by subscriptionid,orderdate)
      --group by 1,2 order by 3 desc
      

""")

┌─────────────────────────┬──────────────────────────────┬──────────────────────────────────────┬──────────────────────────┬──────────────────────┬────────────────┬───────────────────────┬──────────────────────────────────────┬───────────┬───────────────┬────────────┬──────────┬────────────┬──────────────────────┐
│   customerDomainName    │          orderdate           │            subscriptionId            │ truncatedchargestartdate │    chargeEndDate     │   chargeType   │ absEffectiveUnitPrice │             referenceId              │    tfc    │      mn       │ actualqty1 │ totalqty │  newSDate  │       newEDate       │
│         varchar         │           varchar            │               varchar                │           date           │       varchar        │    varchar     │        double         │               varchar                │  double   │    varchar    │   double   │  double  │    date    │       varchar        │
├─────────────────────────┼─────────────────────────

In [17]:
query("select * from onetime where subscriptionid = '4964bb04-cb63-48de-dd99-1cf8aaca4472' order by orderdate")

┌──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────┬─────────────────────────┬─────────────────┬───────────────┬─────────┬───────────────┬──────────────┬──────────────────────────────┬──────────────┬─────────┬────────────────┬──────────────────────────┬──────────────────────────┬───────────────────┬──────────────┬───────────┬────────────────────┬──────────┬──────────┬──────────┬──────────┬──────────────────┬──────────┬───────────────────────┬─────────────┬─────────────────────────┬──────────────────────────────────────┬───────────────────────┬──────────────────────┬──────────────────────────────┬──────────────────────┬──────────────────────────────────────────┬──────────────┬──────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────┬─────────────────┬────────────────────┬

In [18]:
# cxn.close()

In [39]:
query("select min(chargestartdate), max(chargeenddate),count(1) from onetime where productid like 'CFQ%'")

┌──────────────────────┬──────────────────────┬──────────┐
│ min(chargestartdate) │  max(chargeenddate)  │ count(1) │
│       varchar        │       varchar        │  int64   │
├──────────────────────┼──────────────────────┼──────────┤
│ 2023-01-01T00:00:00Z │ 2024-04-21T00:00:00Z │    12875 │
└──────────────────────┴──────────────────────┴──────────┘



In [48]:
cxn.query("""

create or replace temp table onetime_temp as select * from onetime where productid like 'CFQ%' 


""")

In [49]:
cxn.query("""
create or replace  temp table t1 as select * from onetime_temp where chargetype in ('new','renew','cycleCharge','addQuantity','removeQuantity') 

""")

In [104]:
cxn.query("delete from onetime_temp where chargetype in ('new','renew','cycleCharge','addQuantity','removeQuantity')")

In [108]:
requiredColumns = "customerDomainName,subscriptionid,date_trunc('day',cast(chargestartdate as date)) truncatedchargestartdate,chargeenddate,chargetype"

In [128]:
query(f"""
      /*select customerdomainname,max(orderdate),subsctiptionid
      ,max(cast(chargestartdate) as datetime))
      , chargeenddate,billablequantity,monthname 
      from*/
      (
      select {requiredColumns},monthname(cast(subscriptionstartdate as date)) monthname,referenceid 
       -- ,nth_value(billablequantity,2) over (order by orderdate desc) nth
      --,lead(billableQuantity) over (partition by customerdomainname,subscriptionid,truncatedchargestartdate,chargeenddate,chargetype,referenceid,monthname order by orderdate) ld
      --,billablequantity-ld effectiveQty ld
      ,first(billablequantity order by orderdate desc)
      from t1
      where customerdomainname like 'annet%'
       and monthname = 'August'  and subscriptionid = '5e45831b-b5da-4fb0-c743-af024dda5fdb'
        group by customerdomainname,subscriptionid,truncatedchargestartdate,chargeenddate,chargetype,referenceid,monthname
      order by subscriptionid, orderdate
      )
      
      /*  */
      """)

duckdb.duckdb.BinderException('Binder Error: column "orderdate" must appear in the GROUP BY clause or must be part of an aggregate function.\nEither add it to the GROUP BY list, or use "ANY_VALUE(orderdate)" if the exact value of "orderdate" is not important.')

In [129]:
query(f"""select {requiredColumns},monthname(cast(subscriptionstartdate as datetime)) mn,orderdate
       from t1
      where customerdomainname like 'annet%'
       and mn = 'August'  
      and subscriptionid = '5e45831b-b5da-4fb0-c743-af024dda5fdb'
         group  by customerdomainname,subscriptionid,chargestartdate,chargeenddate,chargetype,referenceid,mn
      order by orderdate asc
      """)

duckdb.duckdb.BinderException('Binder Error: column "orderdate" must appear in the GROUP BY clause or must be part of an aggregate function.\nEither add it to the GROUP BY list, or use "ANY_VALUE(orderdate)" if the exact value of "orderdate" is not important.')

In [66]:
query("select sub(num) from (select 1 as num union all select 3 as num)")

duckdb.duckdb.CatalogException('Catalog Error: Scalar Function with name sub does not exist!\nDid you mean "sum"?')